In [34]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Import estimators and transformers
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from src.features import CustomEncoder
from sklearn.decomposition import PCA

# Import predictors
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

# Import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

%load_ext autoreload
%autoreload 2

path_processed = '/home/matteo@COPPET/Documents/data_science/projects/housing_prices_firenze/data/processed/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv(path_processed+'data_clean.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8714 entries, 0 to 8713
Data columns (total 57 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Tipologia                                  8714 non-null   object 
 1   Tipo_proprietà                             8277 non-null   object 
 2   Zona                                       8714 non-null   object 
 3   Anno_costruzione                           6334 non-null   float64
 4   Stato                                      8483 non-null   object 
 5   Prezzo_EUR                                 8714 non-null   float64
 6   Superficie_m2                              8714 non-null   float64
 7   Prezzo_per_m2                              8714 non-null   float64
 8   Riscaldamento_A_C                          8714 non-null   object 
 9   Tipo_riscaldamento                         6760 non-null   object 
 10  Alimentazione_riscaldame

In [3]:
df['Superficie_m2'].describe()

count   8714.000
mean     118.759
std      180.692
min        5.000
25%       65.000
50%       90.000
75%      129.000
max     8150.000
Name: Superficie_m2, dtype: float64

### 1. Outliers

In [4]:
# Remove all rows containing outliers in either Price or Square meters by using z statistic
df = df.loc[(np.abs(stats.zscore(df[['Prezzo_EUR', 'Superficie_m2']])) < 3).all(axis=1)]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8604 entries, 0 to 8713
Data columns (total 57 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Tipologia                                  8604 non-null   object 
 1   Tipo_proprietà                             8174 non-null   object 
 2   Zona                                       8604 non-null   object 
 3   Anno_costruzione                           6265 non-null   float64
 4   Stato                                      8378 non-null   object 
 5   Prezzo_EUR                                 8604 non-null   float64
 6   Superficie_m2                              8604 non-null   float64
 7   Prezzo_per_m2                              8604 non-null   float64
 8   Riscaldamento_A_C                          8604 non-null   object 
 9   Tipo_riscaldamento                         6684 non-null   object 
 10  Alimentazione_riscaldame

### 1. Feature selection and splitting data

In [6]:
target = 'Prezzo_EUR'
#drop_cols = ['Prezzo_per_m2', 'Anno_costruzione', 'Tipo_riscaldamento', 'Alimentazione_riscaldamento',
#             'Classe_energetica'] + [target]

drop_cols = ['Prezzo_per_m2'] + [target]

X = df.drop(columns=drop_cols)
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 2. Pre-processing pipeline

In [7]:
cat_features = ['Tipologia', 'Zona', 'Stato', 'Tipo_proprietà', 'Riscaldamento_A_C', 'Tipo_riscaldamento',
                'Alimentazione_riscaldamento', 'Classe_energetica', 'Piano']
num_features = ['Superficie_m2', 'Num_bagni', 'Num_tot_locali', 'Anno_costruzione']

# cat_features = ['Tipologia', 'Zona', 'Tipo_proprietà', 'Tipo_riscaldamento', 'Classe_energetica']
# num_features = ['Superficie_m2', 'Num_tot_locali']

cat_transformer = Pipeline([
    ('imputing', SimpleImputer(strategy='most_frequent')),
    ('oh_encoding', OneHotEncoder(handle_unknown='ignore'))
])

num_transformer = Pipeline([
    ('scaling', StandardScaler()),
    ('imputing', SimpleImputer(strategy='mean'))
])

preprocessing_pipeline = ColumnTransformer([
    ('categoricals', cat_transformer, cat_features),
    ('numericals', num_transformer, num_features)
],
    remainder='passthrough'
)

### 3. Modeling

#### 3.1 First model (benchmarking)

In [8]:
pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', LinearRegression())
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))

Training set score: 87982.93664249642
Mean cross validation score: 12629211369340.676


#### 3.2 Other models

In [9]:
pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', Ridge())
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))

Training set score: 87950.88673779272
Mean cross validation score: 90305.777493052


#### 3.3 Ensemble models

In [ ]:
# Random Forest
pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', RandomForestRegressor(random_state=0))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))

In [30]:
# Extra-Trees
pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', ExtraTreesRegressor(random_state=0))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))

Training set score: 42.45241900334157
Mean cross validation score: 70487.8029873806


In [35]:
# Gradient boosting
pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', GradientBoostingRegressor(random_state=0))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))

Training set score: 71161.0089714164
Mean cross validation score: 77680.25982218122


In [36]:
# AdaBoost
pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', AdaBoostRegressor(random_state=0))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))

Training set score: 210330.44407089966
Mean cross validation score: 239306.44272788297


The results are much better when removing outliers from the data!

### 4. Hyperparameter tuning

In [31]:
pipe = Pipeline([
    ('preprocessin# Extra-Trees
pipe = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', ExtraTreesRegressor(random_state=0))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))g', preprocessing_pipeline),
    ('model', ExtraTreesRegressor(random_state=0))
])

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 500, num = 4)]
# Number of features to consider at every split
max_features = [2, 3, 4]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 5)]
max_depth.append(None)
# Split criterion
criterion = ['mse', 'mae']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
#param_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'criterion': criterion,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}

param_grid = {
    'model__n_estimators': [100, 200, 300, 400],
    'model__max_features': [2, 4, 8],
    'model__min_samples_split': [2, 4, 6],
    'model__min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(-mean_score, params)

105679.36074126922 {'model__max_features': 2, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 100}
105149.81891998144 {'model__max_features': 2, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
105026.92910200488 {'model__max_features': 2, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 300}
104881.33492112563 {'model__max_features': 2, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 400}
112304.25508193478 {'model__max_features': 2, 'model__min_samples_leaf': 1, 'model__min_samples_split': 4, 'model__n_estimators': 100}
111887.96711604143 {'model__max_features': 2, 'model__min_samples_leaf': 1, 'model__min_samples_split': 4, 'model__n_estimators': 200}
111995.22402172655 {'model__max_features': 2, 'model__min_samples_leaf': 1, 'model__min_samples_split': 4, 'model__n_estimators': 300}
112018.1265625401 {'model__max_features': 2, 'model__mi

In [32]:
grid_search.best_params_

{'model__max_features': 8,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__n_estimators': 400}

In [33]:
pipe = grid_search.best_estimator_

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)

# Train set score
print('Training set score: {}'.format(mean_absolute_error(y_train, y_train_pred)))

# Cross validation score
scores = cross_val_score(pipe, X_train, y_train, scoring='neg_mean_absolute_error', cv=10)
print('Mean cross validation score: {}'.format(np.mean(-scores)))

Training set score: 97.43123153663613
Mean cross validation score: 86277.59267109421
